## Quantum Walk Operators: Circle with 4 Nodes and Line with 16 Nodes

Introduction
Quantum walks are the quantum analog of classical random walks, providing a useful framework for developing quantum algorithms. They have applications in quantum search, graph algorithms, and simulations. In this notebook, we will design quantum walk operators for two specific cases: a circle with 4 nodes and a line with 16 nodes.

Quantum Walk on a Circle with 4 Nodes
A circle (or cyclic graph) with 4 nodes is a simple structure where each node is connected to two others, forming a closed loop. The transition probabilities are equal for each step.

  0 - 1
  |   |
  3 - 2


Transition Probabilities:
Each node has a 50% probability of transitioning to either of its adjacent nodes.

Quantum Walk Operator:

Initialization:

We initialize a quantum register to represent the position of the walker on the graph.
Apply the Hadamard transform to create a superposition of all possible starting positions.
W Operator (Walk Operator):

For each possible state (vertex), we calculate the transition probabilities to the adjacent nodes.
Use controlled operations to update the state based on these probabilities.
S Operator (Shift Operator):

We use a conditional swap operation to simulate the movement between nodes based on the calculated transition probabilities.
Mathematical Representation:

The state space is spanned by the basis states |0⟩, |1⟩, |2⟩, |3⟩.
The walk operator W updates the amplitudes based on the adjacency matrix of the graph.
The shift operator S swaps the states according to the adjacency relationships.
Quantum Walk on a Line with 16 Nodes
A line (or path graph) with 16 nodes is a linear structure where each node (except the endpoints) is connected to two others.

  0 - 1 - 2 - ... - 14 - 15


Transition Probabilities:

For nodes 0 and 15 (endpoints), the transition probability is 100% to their single adjacent node.
For all other nodes, the transition probability is 50% to each adjacent node.
Quantum Walk Operator:

Initialization:

Initialize a quantum register to represent the position of the walker on the graph.
Apply the Hadamard transform to create a superposition of all possible starting positions.
W Operator (Walk Operator):

For each possible state (vertex), we calculate the transition probabilities to the adjacent nodes.
Use controlled operations to update the state based on these probabilities.
S Operator (Shift Operator):

We use a conditional swap operation to simulate the movement between nodes based on the calculated transition probabilities.
Mathematical Representation:

The state space is spanned by the basis states |0⟩, |1⟩, |2⟩, ..., |15⟩.
The walk operator W updates the amplitudes based on the adjacency matrix of the graph.
The shift operator S swaps the states according to the adjacency relationships.

Theoretical and Mathematical Reasoning
Quantum Walk Operators:
The quantum walk operator can be broken down into two primary components:

Walk Operator (W):

Responsible for determining the probabilities of transitioning from one node to another.
Uses controlled operations to apply these transitions based on the current state of the walker.
Shift Operator (S):

Responsible for executing the transitions determined by the walk operator.
Uses conditional swaps to simulate the movement of the walker on the graph.
Hadamard Transform:
The Hadamard transform is applied to create an equal superposition of all possible starting positions. This ensures that the quantum walk explores all nodes simultaneously, leveraging quantum parallelism.

Controlled Operations:
Controlled operations are used to apply the transitions and swaps conditionally, based on the current state of the walker. This allows for the precise control needed to simulate the quantum walk.

Adjacency Matrix:
The adjacency matrix of the graph defines the connections between nodes. It is used to determine the transition probabilities and control the operations accordingly.

In [11]:
from classiq import *

size = 2 #(2^size number of vertices)

@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)


def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0,0,0,0]
    prob[(i+1)% 4]=0.5
    prob[(i-1)% 4]=0.5
    print(f'State={i}, prob vec ={prob}')

    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) ==1)


@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))


@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)

State=0, prob vec =[0, 0.5, 0, 0.5]
State=1, prob vec =[0.5, 0, 0.5, 0]
State=2, prob vec =[0, 0.5, 0, 0.5]
State=3, prob vec =[0.5, 0, 0.5, 0]


In [6]:
import classiq
from classiq import *
from classiq.execution import ExecutionDetails, ExecutionPreferences, ClassiqBackendPreferences
import matplotlib.pyplot as plt
from classiq.execution import ExecutionPreferences
from classiq.qmod.symbolic import floor, log, pi, ceiling

size = 4  # qubits

In [10]:
from classiq import *

size = 4

@qfunc
def prepare_minus(x: QBit):
    X(x)
    H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
    aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
    aux = QNum('aux')
    allocate(1,aux)
    within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):

    prob = [0]* 2**size
    if i==0:
        prob[i+1] = 1.0 #Probability
    elif i==((2**size)-1):
        prob[i-1] = 1.0 #Probability
    else:
        prob[i-1] = 0.5 #Probability
        prob[i+1] = 0.5 #Probability
    print(f'State={i}, prob vec ={prob}')


    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
        W_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    res |= (((vertices-adjacent_vertices) == 1) | ((vertices-adjacent_vertices) == -1))


@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
    repeat(count= x.len,
        iteration= lambda i: SWAP(x[i],y[i]))


@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    '''function'''

    allocate(size,vertices)
    hadamard_transform(vertices)
    allocate(size,adjacent_vertices)

    W_operator(vertices,adjacent_vertices)
    S_operator(vertices,adjacent_vertices)

qmod = create_model(main)


State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0